In [35]:
# https://www.kaggle.com/c/nlp-getting-started/datahttps://www.kaggle.com/c/nlp-getting-started/data
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>=0.99):
          self.model.stop_training = True

In [36]:
train_set = pd.read_csv("data/real_or_not_NLP_challenge/train.csv",sep=",", encoding="ISO-8859-1", low_memory=False,)
test_set = pd.read_csv("data/real_or_not_NLP_challenge/test.csv",sep=",", encoding="ISO-8859-1", low_memory=False,)

tokeninzer_train = Tokenizer()
tokeninzer_train.fit_on_texts(train_set.text)
x_train = pad_sequences(tokeninzer_train.texts_to_sequences(train_set.text), padding='post',truncating='post', maxlen=20)
y_train = train_set.target.array.to_numpy()
vocab_size = len(tokeninzer_train.word_index)*2

tokeninzer_test = Tokenizer()
tokeninzer_test.fit_on_texts(test_set.text)
x_test = pad_sequences(tokeninzer_test.texts_to_sequences(test_set.text), padding='post',truncating='post', maxlen=20)

In [ ]:
model = tf.keras.Sequential([ 
  tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=32, input_length=20),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
  tf.keras.layers.Dense(64, activation=tf.keras.activations.relu),
  tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid) ])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
history = model.fit(x=x_train, y=y_train, epochs=10, callbacks=[myCallback()])

In [48]:
predictions = model.predict(x_test)
test_set["target"] = (predictions > 0.5).astype(int)
test_set

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",0
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,0
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÂÃÃ SAFETY FA...,1
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,1
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,0
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,1


In [50]:
train_set

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
